## Anna Kaniowska - Cellphones & Accessories dataset analysis

The goal of this project is to extract as much information form the data set that can be obtained here - http://snap.stanford.edu/data/amazon/Cell_Phones_&_Accessories.txt.gz (source: http://snap.stanford.edu/data/web-Amazon-links.html)

In [1]:
# All imports needed to perform the analysis
import gzip
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### Loading the dataset

In [2]:
# A modified version of function available on the source page
def parse(filename):
    """
    Parses a gzipped text file and returns a list of dictionaries containing the parsed data.

    Parameters:
    filename (str): The path to the gzipped text file to be parsed.

    Returns:
    list: A list of dictionaries containing the parsed data.
    """
    f = gzip.open(filename, 'rb')
    entry = {}
    data = []
    for line in f:
        l = line.decode('utf-8').strip()
        colonPos = l.find(':')
        if colonPos == -1:
            data.append(entry)
            entry = {}
            continue
        eName = l[:colonPos]
        rest = l[colonPos+2:]
        entry[eName] = rest
    data.append(entry)
    return data

# Loading the dataset
data = parse('Cell_Phones_&_Accessories.txt.gz')
df = pd.DataFrame(data)


#### Getting to know the dataset

In [3]:
df.head(10)

,product/productId,product/title,product/price,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,B000JVER7W,Mobile Action MA730 Handset Manager - Bluetoot...,unknown,A1RXYH9ROBAKEZ,A. Igoe,0/0,1.0,1233360000,Don't buy!,"First of all, the company took my money and se..."
1,B000JVER7W,Mobile Action MA730 Handset Manager - Bluetoot...,unknown,A7L6E1KSJTAJ6,Steven Martz,0/0,5.0,1191456000,Mobile Action Bluetooth Mobile Phone Tool Soft...,Great product- tried others and this is a ten ...
2,B000JVER7W,Mobile Action MA730 Handset Manager - Bluetoot...,unknown,A2NOQQ7I6PU7CW,"Daniel M. Johnson ""rocknbluesharp""",0/0,4.0,1186704000,good,works real good....a little hard to set up...w...
3,B000924R5I,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",unknown,A29Q0CLOF0U8BN,E. Owens,4/5,4.0,1146182400,No instructions included...,The price was right for this cable ($11.95+$4....
4,B000924R5I,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",unknown,ANNHJND2MHUI3,"Isaac Salas ""=CRBF=gB^link""",0/0,1.0,1173657600,NOT A DATA CABLE,this is NOT a DATA CABLE this is only a USB ch...
5,B000924R5I,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",unknown,A2DP6EPPQAIL3C,David M. Cantrell,0/0,4.0,1171584000,works as expected,There's not much to say about a data cable -- ...
6,B000924R5I,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",unknown,A254KU5H615J3N,J. S. Gaviota,0/0,5.0,1142467200,Excellent product!,"I have a Sony Ericsson K700i, which doesn't se..."
7,B000924R5I,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",unknown,ACVTUAQ4RUGK,"Parathalyn ""Parathalyn""",0/0,1.0,1133654400,Beware 3rd Party Cables & Sony Ericsson Tech S...,"Well, it technically does what it is supposed ..."
8,B000924R5I,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",unknown,A34D9XVHS261KT,D. Cram,1/2,3.0,1135814400,"Undecided on product, but Electro Island is A+...",I currently have this item for my S/E z500a ph...
9,B000924R5I,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",unknown,A3Q3B32YUDC3IK,"Jonathan C. Phillips ""Chris""",6/10,4.0,1126915200,Good cable to have,This is a good cable to have. You don't need t...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78931 entries, 0 to 78930
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   product/productId   78930 non-null  object
 1   product/title       78930 non-null  object
 2   product/price       78930 non-null  object
 3   review/userId       78930 non-null  object
 4   review/profileName  78930 non-null  object
 5   review/helpfulness  78930 non-null  object
 6   review/score        78930 non-null  object
 7   review/time         78930 non-null  object
 8   review/summary      78930 non-null  object
 9   review/text         78930 non-null  object
dtypes: object(10)
memory usage: 6.0+ MB


In [5]:
df.describe()

,product/productId,product/title,product/price,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
count,78930,78930,78930,78930,78930,78930,78930,78930,78930,78930
unique,7438,7150,944,68041,63370,1305,5,4431,60321,77390
top,B0009B0IX4,Plantronics Voyager 510 Bluetooth Headset [Ret...,unknown,unknown,unknown,0/0,5.0,1168300800,Great Product,I had been very happy with my Motorola phone u...
freq,3443,3443,43287,2276,2276,34920,30253,519,211,20


In [6]:
print(f"The shape of the dataset is: {df.shape}")

The shape of the dataset is: (78931, 10)


Taking a first look at the data, it is visible that it shows the reviews that customers gave to the products. The products are mainly cellphones and their accesories. The dataset is big - almost 79 000 rows is a significant amount of data. 10 columns provide information about the rated product, the customer and their opinion on the product. When it comes to technical details - it is necessary to change 'unknown' values to NaN in order to prepare data to further analysis. Checking the dataset for duplicated rows and dropping existing ones is also necessary because this is something that cannot be seen at first glance. Another conclusion is that the types of the columns are not necessarily correct (e.g. product/price should be stored as float), it is also needed to be corrected.

#### Checking for duplicated rows

In [7]:
print(f"There is {df.duplicated().sum()} duplicated rows in the dataset")

There is 36 duplicated rows in the dataset


In [8]:
df.drop_duplicates(inplace=True)

#### Handling missing values

In [9]:
df.replace("unknown", np.nan, inplace=True)

# Analyzing the missing values occurences
print("Missing values occurences:")
print(df.isna().sum())

# Checking anonymous reviews (those where userId and profileName is missing)
anon_reviews_perc = df['review/userId'].isna().sum()/df.shape[0] * 100
print(f"{anon_reviews_perc:.2f}% of the reviews are anonymously submitted.")


Missing values occurences:
product/productId         1
product/title             1
product/price         43266
review/userId          2275
review/profileName     2275
review/helpfulness        1
review/score              1
review/time               1
review/summary            1
review/text               1
dtype: int64
2.88% of the reviews are anonymously submitted.


The first conclusion is one row that can be safely deleted in each column (it is very likely that it is the same row for each of the columns). \
The second conclusion refers to anonymous reviews. When missing values are less than 5% of given feature, they can be safely deleted without having impact on further analysis. For now the data will be divided into to sets - first with anonymous reviews and second with named reviews. It may be useful to further analysis.

In [10]:
# Extracting columns with 1 missing value
cols = ['product/productId', 'product/title', 'review/helpfulness', 'review/score', 'review/time', \
        'review/summary', 'review/text']
df = df.dropna(subset=cols)
print(f"Shape of the dataset after dropping NaNs: {df.shape}")

Shape of the dataset after dropping NaNs: (78894, 10)


As expected, only one row of the data was deleted.

Before dividing dataset into two separate dataset it would be useful to correct the columns' types.

#### Correcting columns types

In [11]:
df['product/price'] = pd.to_numeric(df['product/price'], errors='coerce')
df['review/score'] = pd.to_numeric(df['review/score'], errors='coerce')
df['review/time'] = pd.to_datetime(df['review/time'].astype(float), unit='s')

def handle_helpfulness(x):
    """
    Converts the string representation of helpfulness scores to a float value between 0 and 1.

    Parameters:
        helpfulness (str): The string representation of helpfulness scores, in the format "x/y",
        where "x" is the number of users who found the review helpful and "y" is the total number of votes.

    Returns:
        float: The float value of the helpfulness score, calculated as "x / y". Returns 0 if "y" is 0.
    """
    try:
        nom, denom = x.split("/")
        return int(nom) / int(denom)
    except (ValueError, ZeroDivisionError):
        return 0

df['review/helpfulness'] = df['review/helpfulness'].apply(handle_helpfulness)

The price and score columns are stroing numeric values, the review has time in seconds. When it comes to helpfulness it was transformed to the float value that represents it.

#### Division of dataset

In [12]:
# Dividing the dataset into anonymous and named
df_anon = df[(df['review/userId'].isna()) & (df['review/profileName'].isna())]
df_named = df[~(df['review/userId'].isna()) & ~(df['review/profileName'].isna())]

print(f"Missing values occurences in anonymous reviews dataset:\n{df_anon.isna().sum()}")
print(f"Missing values occurences in named reviews dataset:\n{df_named.isna().sum()}")

Missing values occurences in anonymous reviews dataset:
product/productId        0
product/title            0
product/price         1998
review/userId         2274
review/profileName    2274
review/helpfulness       0
review/score             0
review/time              0
review/summary           0
review/text              0
dtype: int64
Missing values occurences in named reviews dataset:
product/productId         0
product/title             0
product/price         41267
review/userId             0
review/profileName        0
review/helpfulness        0
review/score              0
review/time               0
review/summary            0
review/text               0
dtype: int64


The last thing about missing values is the price. As it is a significant amount of data in both datasets it would not make sense to drop it. After seeing more details of the data a desicion will be made whether to replace the missing values with mean, mode or median or to create a new category for them.

#### Data Visualizations